In [ ]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install  'sklearn'  
!pip3 install --upgrade 'pillow'

**Import libraries**

In [ ]:
import os
import logging
import sys
import io
import copy 
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torchvision.models import vgg11

from PIL import Image
from tqdm import tqdm

from torchvision.datasets import VisionDataset

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

**Set Arguments**

In [ ]:
DEVICE = 'cuda'                # 'cuda' or 'cpu'

NUM_CLASSES = 101              # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256               # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                               # the batch size, learning rate should change by the same factor to have comparable results

LRS = [0.1, 0.01, 0.01, 0.001 ]                  # The initial Learning Rate
MOMENTUM = 0.9                                   # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5                              # Regularization, you can keep this at the default

NUM_EPOCHS = 200                        # Total number of training epochs (iterations over dataset)
STEP_SIZE = [20, 30, 45, 60]             # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1                              # Multiplicative factor for learning rate step-down


#Values to early stopping
HP = 5                    
strip_len = 5

# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      
                                      transforms.CenterCrop(224),  
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

# Define transforms for the alexnet pretrained
TL_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))                                    
])

# Define tranforms for the data augmentation
data_augmentation_transform_1 = transforms.Compose([transforms.Resize(256), 
                                      transforms.RandomCrop(224),  
                                      transforms.Grayscale(3),  
                                      transforms.RandomHorizontalFlip(),          
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  
])

data_augmentation_transform_2 = transforms.Compose([transforms.Resize(256),      
                                      transforms.Grayscale(3),  
                                      transforms.RandomCrop(224),  
                                      transforms.RandomVerticalFlip(),          
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  
])

data_augmentation_transform_3 = transforms.Compose([transforms.Resize(256),      
                                      transforms.RandomCrop(224),  
                                      transforms.RandomHorizontalFlip(),          
                                      transforms.ColorJitter(),                                                                                
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  
])

**Utility**




In [ ]:
def makePlot(x, ys, xLabel, yLabel, legend):
    for y in ys:
        plt.plot(x, y)
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)
    plt.legend(legend)
    plt.show()


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    img = Image.open(open(path, 'rb'), mode='r')
    return img.convert('RGB')


def make_dataset(dir, class_to_idx, extensions=None, is_valid_file=None):
    images = []
    dir = os.path.expanduser(dir)
    if not ((extensions is None) ^ (is_valid_file is None)):
        raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")
    if extensions is not None:
        def is_valid_file(x):
            return has_file_allowed_extension(x, extensions)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d, followlinks=True)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                if is_valid_file(path):
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images


def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (tuple of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    return filename.lower().endswith(extensions)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)


def prepareDataloader(transform):
    # Clone github repository with data
    if not os.path.isdir('./Homework2-Caltech101'):
        !git
        clone
        https: // github.com / MachineLearning2020 / Homework2 - Caltech101.git

    DATA_DIR = 'Homework2-Caltech101/101_ObjectCategories'

    # Prepare Pytorch Datasets
    dataset = Caltech(DATA_DIR, transform=transform)

    train_indexes = []
    test_indexes = []

    test_path = open("Homework2-Caltech101/test.txt", "r")

    names = []


for name in test_path.readlines():
    names.append(name[:-1])

num_lines = len(open("Homework2-Caltech101/test.txt").readlines())

for index in range(len(dataset)):
    test = False
    for name in names:
        if (DATA_DIR + "/" + name) == dataset.samples[index][0]:
            test_indexes.append(index)
            test = True
            test_path.seek(0)
            break
test_path.seek(0)

if not test:
    train_indexes.append(index)

train_val_dataset = Subset(dataset, train_indexes)
test_dataset = Subset(dataset, test_indexes)

train_dataset = []
val_dataset = []

# Essendo gli elementi ordinati per classe va bene anche questo split
for index in range(len(train_val_dataset)):
    if (index % 2) == 0:
        train_dataset.append(train_val_dataset[index])
    else:
        val_dataset.append(train_val_dataset[index])

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Val Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))
print('Train + Val dataset: {}'.format(len(train_val_dataset)))

# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
train_val_dataloader = DataLoader(train_val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

return train_dataloader, val_dataloader, test_dataloader, train_val_dataloader


def stoppingCriteria(accuracy, last_acc_val, hp, early_stopping, epoch):
    if accuracy > last_acc_val:
        hp = HP
        aux_str = ""
    else:
        hp -= 1
        aux_str = " Accuracy has not encreased, hp=" + str(hp)

    if epoch >= (strip_len * HP) and hp == 0:
        early_stopping = True

        last_acc_val = accuracy
        return aux_str, early_stopping, hp, last_acc_val


def tuneHyperparameters(train_dataloader, val_dataloader, test_dataloader, train_val_dataloader, input_net):
    ##############################
    #   Tuning Hyperparameters   #
    ##############################

    top_model = {'lr': 0, 'num_epoch': 0, 'acc': 0, 'step_size': 0}

    for i in range(len(LRS)):
        print("Starting (LR=" + str(LRS[i]) + ", step size=" + str(STEP_SIZE[i]) + ").")

        net = copy.deepcopy(input_net)
        net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

        criterion = nn.CrossEntropyLoss()
        parameters_to_optimize = net.parameters()

        optimizer = optim.SGD(parameters_to_optimize, lr=LRS[i], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE[i], gamma=GAMMA)

        net = net.to(DEVICE)
        epoch = 0
        early_stopping = False
        hp = copy.deepcopy(HP)
        last_acc_val = 0
        top_val_partial = {'acc': 0, 'epoch': 0}

    while (epoch < NUM_EPOCHS and early_stopping == False):
        epoch += 1

        for images, labels in train_dataloader:

            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            net.train()
            optimizer.zero_grad()  # Zero-ing the gradients

            outputs = net(images)
            loss = criterion(outputs, labels)
            if math.isnan(loss):
                print("           loss:", loss)
                early_stopping = True

            loss.backward()  # backward pass: computes gradients
            optimizer.step()  # update weights based on accumulated gradients

        scheduler.step()

        ################################################
        #   Evaluate the model on the validation set   #
        ################################################

    net.train(False)  # Set Network to evaluation mode
    running_corrects = 0

    for images, labels in val_dataloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = net(images)

        # Get predictions
        _, preds = torch.max(outputs.data, 1)

        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()

    # Calculate Accuracy
    accuracy = running_corrects / float(len(val_dataloader.dataset))

    if accuracy > top_model['acc']:
        top_model['acc'] = accuracy
        top_model['lr'] = LRS[i]
        top_model['num_epoch'] = epoch
        top_model['step_size'] = STEP_SIZE[i]

    aux_str = ""
    if epoch % strip_len == 0:
        aux_str, early_stopping, hp, last_acc_val = stoppingCriteria(accuracy, last_acc_val, hp, early_stopping, epoch)

    if top_val_partial['acc'] < accuracy:
        top_val_partial['acc'] = accuracy
        top_val_partial['epoch'] = epoch

    print(" Ending epoch " + str(epoch) + "/" + str(NUM_EPOCHS) + ", LR = " + str(
        scheduler.get_lr()) + ", step size = " + str(STEP_SIZE[i]) + ", Accuracy on validation set = " + str(
        round(accuracy * 100, 4)) + aux_str)


print("Best accuracy: ", round(top_val_partial['acc'] * 100, 4), "%. Epoch: ", top_val_partial['epoch'], ".")

print("The set of the best hyperparameters is: [lr = " + str(top_model["lr"]) +
      ", num_epoch = " + str(top_model['num_epoch']) + ", step size = " +
      str(top_model['step_size']) + " ]. The best accuracy is: " +
      str(round(top_model['acc'] * 100, 2)) + "%.")

#############################
#   Train net on train+val  #
#############################

print(
    "----------------------------------------------------------------------------------------------------------------------------------------------")
print("Strart train net on train+val")

net = copy.deepcopy(input_net)
net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

criterion = nn.CrossEntropyLoss()
parameters_to_optimize = net.parameters()

optimizer = optim.SGD(parameters_to_optimize, lr=top_model['lr'], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=top_model['step_size'], gamma=GAMMA)

net = net.to(DEVICE)

for epoch in range(top_model['num_epoch']):

    for images, labels in train_dataloader:

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        net.train()
        optimizer.zero_grad()  # Zero-ing the gradients

        outputs = net(images)
        loss = criterion(outputs, labels)
        if math.isnan(loss):
            print("           loss:", loss)

        loss.backward()  # backward pass: computes gradients
        optimizer.step()  # update weights based on accumulated gradients

    scheduler.step()
    print(" Ending epoch " + str(epoch + 1) + "/" + str(top_model['num_epoch']) + ", LR = ", str(scheduler.get_lr()),
          ", step size = ", top_model['step_size'])

#########################
#   Evaluate Test Set   #
#########################

print(
    "----------------------------------------------------------------------------------------------------------------------------------------------")
print("Strart test net on test")
net = net.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
net.train(False)  # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataloader.dataset))

print("Test Accuracy: " + str(round(accuracy * 100, 2)) + "%.")

return top_model


def partialTrain(train_val_dataloader, test_dataloader, top_model_full_opt, opt):
    top_model = {'net': 0, 'num_epoch': 0, 'acc': 0}
    print("Starting (LR=" + str(top_model_full_opt["lr"]) + ", step size=" + str(
        top_model_full_opt["step_size"]) + ", num max epoch: " + str(top_model_full_opt["num_epoch"]) + ").")

    net = alexnet(pretrained=True)
    net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

    criterion = nn.CrossEntropyLoss()

    if opt == "FC":
        parameters_to_optimize = net.classifier.parameters()
        parameters_to_not_optimize = net.features.parameters()
        for p in parameters_to_not_optimize:
            p.requires_grad = False
    else:
        parameters_to_optimize = net.features.parameters()
        parameters_to_not_optimize = net.classifier.parameters()
        for p in parameters_to_not_optimize:
            p.requires_grad = False

    optimizer = optim.SGD(parameters_to_optimize, lr=top_model_full_opt["lr"], momentum=MOMENTUM,
                          weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=top_model_full_opt["step_size"], gamma=GAMMA)

    net = net.to(DEVICE)

    for epoch in range(top_model_full_opt['num_epoch']):

        ##############################
        #   Train Net on train+val   #
        ##############################

        for images, labels in train_val_dataloader:

            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            net.train()
            optimizer.zero_grad()  # Zero-ing the gradients

            outputs = net(images)
            loss = criterion(outputs, labels)
            if math.isnan(loss):
                print("           loss:", loss)

            loss.backward()  # backward pass: computes gradients
            optimizer.step()  # update weights based on accumulated gradients

        scheduler.step()
        print(" Ending epoch " + str(epoch) + "/" + str(top_model_full_opt['num_epoch']) + ", LR = ",
              top_model_full_opt['lr'], ", step size = ", top_model_full_opt['step_size'])

    #########################
    #   Evaluate Test Set   #
    #########################

    net = net.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
    net.train(False)  # Set Network to evaluation mode

    running_corrects = 0
    for images, labels in tqdm(test_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = net(images)

        # Get predictions
        _, preds = torch.max(outputs.data, 1)

        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()

    # Calculate Accuracy
    accuracy = running_corrects / float(len(test_dataloader.dataset))

    print("Test Accuracy: " + str(round(accuracy * 100, 2)) + "%.")


class Caltech(VisionDataset):

    def __init__(self, root, split='train', transform=None, target_transform=None):

        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

        classes, class_to_idx = self._find_classes(self.root)
        samples = make_dataset(self.root, class_to_idx, IMG_EXTENSIONS)

        if len(samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                                                                 "Supported extensions are: " + ",".join(
                IMG_EXTENSIONS)))

        self.split = split  # This defines the split you are going to use
        # (split files are called 'train.txt' and 'test.txt')
        self.loader = default_loader
        self.extensions = IMG_EXTENSIONS

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]
        self.transform = transform
        self.target_transform = target_transform

    def _find_classes(self, dir):
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        classes.remove("BACKGROUND_Google")
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
        # Provide a way to access image and label via index. Image should be a PIL Image  label can be int
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    def __len__(self):
        length = len  # Provide a way to get the length (number of elements) of the dataset
        return len(self.samples)

**Punti 1, 2**





In [ ]:
%xmode Verbose
train_dataloader, val_dataloader, test_dataloader, train_val_dataloader = prepareDataloader(train_transform)
net = alexnet() 
tuneHyperparameters(train_dataloader, val_dataloader, test_dataloader, train_val_dataloader, net)

**Transfert Learning**
Punto 3



In [ ]:
%xmode Verbose
#################
#   Punto 3.A   #
#################
net = alexnet(pretrained = True)


#################
#   Punto 3.B   #
#################
train_dataloader_TL, val_dataloader_TL, test_dataloader_TL, train_val_dataloader_TL = prepareDataloader(TL_transform)


#################
#   Punto 3.C   #
#################
print("#################")
print("#   Punto 3.C   #")
print("#################")
top_model_full_opt = tuneHyperparameters(train_dataloader_TL, val_dataloader_TL, test_dataloader_TL, train_val_dataloader_TL, net)


#################
#   Punto 3.D   #
#################
print("#################")
print("#   Punto 3.D   #")
print("#################")
partialTrain(train_val_dataloader_TL, test_dataloader_TL, top_model_full_opt, "FC")


#################
#   Punto 3.E   #
#################
print("#################")
print("#   Punto 3.E   #")
print("#################")
partialTrain(train_val_dataloader_TL, test_dataloader_TL, top_model_full_opt, "CL")

**Punto 4**

In [ ]:

%xmode Verbose
train_dataloader_DA_1, val_dataloader_DA_1, test_dataloader_DA_1, train_val_dataloader_DA_1 = prepareDataloader(data_augmentation_transform_1)
train_dataloader_DA_2, val_dataloader_DA_2, test_dataloader_DA_2, train_val_dataloader_DA_2 = prepareDataloader(data_augmentation_transform_2)
train_dataloader_DA_3, val_dataloader_DA_3, test_dataloader_DA_3, train_val_dataloader_DA_3 = prepareDataloader(data_augmentation_transform_3)

net = alexnet(pretrained = True)

print("\n----------------------------------------------------------------------\n")
print("First DA")
tuneHyperparameters(train_dataloader_DA_1, val_dataloader, test_dataloader, train_val_dataloader_DA_1, net) 
print("\n----------------------------------------------------------------------\n")
print("Second DA")
tuneHyperparameters(train_dataloader_DA_2, val_dataloader, test_dataloader, train_val_dataloader_DA_2, net) 
print("\n----------------------------------------------------------------------\n")
print("Third DA")
tuneHyperparameters(train_dataloader_DA_3, val_dataloader, test_dataloader, train_val_dataloader_DA_3, net) 


Punto 5


In [ ]:
%xmode Verbose

BATCH_SIZE = 32
print("#################")
print("# Punto 5.1/5.2 #")
print("#################")

net = vgg11() 
train_dataloader, val_dataloader, test_dataloader, train_val_dataloader = prepareDataloader(train_transform)
tuneHyperparameters(train_dataloader, val_dataloader, test_dataloader, train_val_dataloader, net)


#################
#   Punto 5.3   #
#################

print("#################")
print("#   Punto 5.3   #")
print("#################")

net = vgg11(pretrained = True)
train_dataloader_TL, val_dataloader_TL, test_dataloader_TL, train_val_dataloader_TL = prepareDataloader(TL_transform)
top_model_full_opt = tuneHyperparameters(train_dataloader_TL, val_dataloader_TL, test_dataloader_TL, train_val_dataloader_TL, net)


#################
#   Punto 5.4   #
#################

print("#################")
print("#   Punto 5.4   #")
print("#################")

train_dataloader_DA_1, val_dataloader_DA_1, test_dataloader_DA_1, train_val_dataloader_DA_1 = prepareDataloader(data_augmentation_transform_1)
train_dataloader_DA_2, val_dataloader_DA_2, test_dataloader_DA_2, train_val_dataloader_DA_2 = prepareDataloader(data_augmentation_transform_2)
train_dataloader_DA_3, val_dataloader_DA_3, test_dataloader_DA_3, train_val_dataloader_DA_3 = prepareDataloader(data_augmentation_transform_3)

print("\n----------------------------------------------------------------------\n")
print("First DA")
tuneHyperparameters(train_dataloader_DA_1, val_dataloader, test_dataloader, train_val_dataloader_DA_1, net) 
print("\n----------------------------------------------------------------------\n")
print("Second DA")
tuneHyperparameters(train_dataloader_DA_2, val_dataloader, test_dataloader, train_val_dataloader_DA_2, net) 
print("\n----------------------------------------------------------------------\n")
print("Third DA")
tuneHyperparameters(train_dataloader_DA_3, val_dataloader, test_dataloader, train_val_dataloader_DA_3, net) 